In [ ]:
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import geopandas as gpd

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [ ]:
## Indicador del tipo de red por horario  
Tipo = "_Hor.csv"
TipoS = "_Hor4.csv"
Condicion = 4

In [ ]:
mexico_city_map = Basemap(llcrnrlon=-99.4, llcrnrlat=19.1, urcrnrlon=-98.8, urcrnrlat=19.7, resolution='l', epsg=4326)
# Cargamos las coordenadas de las estaciones 
ecobici_coordenadas = pd.read_csv("https://raw.githubusercontent.com/JobGV/ECOBICI/main/Dictionary/Cordenadas_Ecobici.csv")
ecobici_coordenadas = ecobici_coordenadas.rename(columns={'id':'source','Latitud':'longitud','Longitud':'latitud'})
pos =ecobici_coordenadas[['source','latitud','longitud']]
# Asignamos las coordenadas 
mx, my =mexico_city_map(ecobici_coordenadas[['longitud']],ecobici_coordenadas[['latitud']])
ecobici_coordenadas[['longitud']]=mx
ecobici_coordenadas[['latitud']] = my
pos['location']=pos[['latitud','longitud']].apply(tuple,axis=1)
pos=pos[['source','location']]
pos_dic = dict(zip(pos['source'], pos['location']))

Year=[2019,2020,2021]

for i in range(3):
    rutaR ="https://raw.githubusercontent.com/JobGV/ECOBICI/main/Clean/"+str(Year[i])+Tipo
    ecobici = pd.read_csv(rutaR)
    
    ecobici = ecobici[ecobici['Horario']==Condicion]
    
    ecobici = ecobici.rename(columns={'Ciclo_Estacion_Retiro':'source','Ciclo_Estacion_Arribo':'target'})
    edges = ecobici[['source', 'target','Viajes']].reset_index(drop=True) 
    edges=edges.rename(columns={'Viajes':'weight'})
    
    # Ajustamos la red 
    G=nx.from_pandas_edgelist(edges,source='source',target='target',edge_attr='weight',create_using=nx.DiGraph())
    nx.set_node_attributes(G, pos_dic, 'coord')
    
    # Seleccionamos el top n
    n=200
    # Calulamos in-degree y out-degree
    in_deg_cent = nx.in_degree_centrality(G)
    out_deg_cent = nx.out_degree_centrality(G)
    in_deg_central = pd.DataFrame(in_deg_cent.items(), columns=['id', 'in_deg_cent'])
    out_deg_central = pd.DataFrame(out_deg_cent.items(), columns=['id', 'out_deg_cent'])
    Top_in = in_deg_central.nlargest(n,'in_deg_cent')
    Top_out = out_deg_central.nlargest(n,'out_deg_cent')

    lst1 = Top_in.id.values.tolist() 
    lst2 = Top_in.in_deg_cent.values.tolist()
    lst3 = ['in_deg_cent']*n
    Centralidades = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns=['id','value','mesure'])

    lst1 = Top_out.id.values.tolist() 
    lst2 = Top_out.out_deg_cent.values.tolist()
    lst3 = ['out_deg_cent']*n
    aux = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns=['id','value','mesure'])

    Centralidades = pd.concat([Centralidades,aux])
    
    # Calculamos la centralidad de cercania 
    close_cent=nx.closeness_centrality(G)
    close_central=pd.DataFrame(close_cent.items(), columns=['id', 'close_cent'])
    Top_close=close_central.nlargest(n,'close_cent')

    lst1 = Top_close.id.values.tolist() 
    lst2 = Top_close.close_cent.values.tolist()
    lst3 = ['close_cent']*n
    aux = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns=['id','value','mesure'])

    Centralidades = pd.concat([Centralidades,aux])

    # Calculamos la centralidad betwenness
    between_cent=nx.betweenness_centrality(G)
    between_central=pd.DataFrame(between_cent.items(), columns=['id', 'between_cent'])
    Top_between=between_central.nlargest(n,'between_cent')

    lst1 = Top_between.id.values.tolist() 
    lst2 = Top_between.between_cent.values.tolist()
    lst3 = ['between_cent']*n
    aux = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns=['id','value','mesure'])

    Centralidades = pd.concat([Centralidades,aux])
    #Calculamos el clustering promedio y el grado de asortatividad
    lst1 = [0]*3
    lst2 = [nx.average_clustering(G),nx.degree_assortativity_coefficient(G),nx.degree_pearson_correlation_coefficient(G)]
    lst3 = ['average_clustering','degree_assortativity','degree_pearson_corr']

    aux = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns=['id','value','mesure'])
    
    # Creamos una base con todos todas las medidas de centralidad
    Centralidades = pd.concat([Centralidades,aux])
    Centralidades['Year']=Year[i]
    #rutaS="C:\\Users\\benga\\OneDrive - El Colegio de México A.C\\Proyecto\\Redes\\Resultados\\Centralidades_"+str(Year[i])+TipoS
    #Centralidades.to_csv(rutaS,index=True)

In [ ]:
# Crear la base consolidada por género 
Year=[2019,2020,2021]
rutaS="https://raw.githubusercontent.com/JobGV/ECOBICI/main/Medidas_Centralidad/Centralidades_"+str(Year[2])+TipoS
base = pd.read_csv(rutaS)

for i in range(2):
    rutaS="https://raw.githubusercontent.com/JobGV/ECOBICI/main/Medidas_Centralidad/Centralidades_"+str(Year[i])+TipoS
    aux = pd.read_csv(rutaS)
    base= pd.concat([base,aux])

Etiquetas = pd.read_csv("https://raw.githubusercontent.com/JobGV/ECOBICI/main/Dictionary/Estaciones.csv")
base = base.merge(Etiquetas,how='left',on='id')
base['nombre'] = base['id'].astype(str)+". "+base['Estación']+" ("+round(base['value'],4).astype(str)+")"
#ruta="C:\\Users\\benga\\OneDrive - El Colegio de México A.C\\Proyecto\\Redes\\Resultados\\Centralidades_Consolidado"+TipoS
#base.to_csv(ruta,index=False)   